In [1]:
import glob

In [2]:
#clean text
import re


In [3]:
from tqdm import tqdm

In [4]:
def clean_text(txt):
    rpl_list=[
    ]
    for r in rpl_list:
        txt=txt.replace(r,"")

    txt=re.sub(' +', ' ', txt)
    return txt.strip()


In [5]:
import json
asked_json_list=glob.glob("database/questions/raw/*.json")
print("asked json list: ",len(asked_json_list))




q_a_list=[]
for path in tqdm(asked_json_list[1:]):
    with open(path) as f:
        asked_json=json.load(f)


    context=""
    a=""
    for idx in range(len(asked_json)):
        #s_text="#text:"
        s_text="You are a scientific textbook"
        q_pos=asked_json[idx]["text"].find(s_text)
        if q_pos>0:
            if context!="":
                a=a[a.find("Q1: "):]

                a=clean_text(a).strip()
                q_a_list.append((context,a))

            #question mode
            context=asked_json[idx]["text"]
            context=context[context.find("#text:")+8:].strip()
            context=clean_text(context)
            a=""
        else:
            #answer mode
            a+=asked_json[idx]["text"]
            



asked json list:  84


100%|██████████| 83/83 [00:11<00:00,  7.03it/s]


In [6]:
#コンテキスト + q a の形式にする
q_a_list2=[]

for context,a in tqdm(q_a_list):
    q_a_series=a.split("\n")
    q_a_series=[i for i in q_a_series if len(i)>3]

    q_id=0
    for line_ in q_a_series:
        line=line_.strip()
        if line.startswith("Q"):
            q=line
            try:
                q_id=int(line[1])
            except:
                continue
        elif line.startswith("A"):
            try:
                a_id=int(line[1])
            except:
                continue
            if q_id==a_id:
                q_a_list2.append((context,q,line))
        else:
            continue

100%|██████████| 92863/92863 [00:00<00:00, 155677.27it/s]


In [7]:
import joblib
q_a_path="database/questions/q_a_list0926.joblib"

In [8]:

joblib.dump(q_a_list2,q_a_path)

['database/questions/q_a_list0926.joblib']

In [9]:
q_a_list2=joblib.load(q_a_path)

In [10]:
len(q_a_list2)

419957

In [11]:
raw_data_list=joblib.load("database/processed/0926.bin")

In [12]:
len(raw_data_list)

65329

In [13]:
idx=0
targt_context=q_a_list2[idx][0]
targt_context

'Dirac, P. A. M. Proc. R. Soc. Lond. A 123, 714 (1929). Jensen, F. Introduction to Computational Chemistry 3rd edn (Wiley, 2017). Shavitt, I.&amp; Bartlett, R. J. Many-Body Methods in Chemistry and Physics. MBPT and Coupled-Cluster Theory (Cambridge Univ. Press, 2009). Mihm, T. N. et al. Nat. Comput. Sci.(2021). Liao, K.&amp; Greneis, A. J. Chem. Phys.145, 141102 (2016). Gruber, T., Liao, K., Tsatsoulis, T., Hummel, F.&amp; Gruneis, A. Phys. Rev. X 8, 021043 (2018). Kent, P. R. C.&amp; Kotliar, G. Science 361, 348 (2018). Knizia, G.&amp; Chan, G. K.-L. Phys. Rev. Lett.109, 186404 (2012). Perdew J.&amp; Kurth, S. A Primer in Density Functional Theory (Springer, 2003). Bogojeski, M., Vogt-Maranto, L., Tuckerman, M., Muller, K.&amp; Burke, K. Nat. Commun.11, 5223 (2020).'

In [14]:
q_a_list2[idx]

('Dirac, P. A. M. Proc. R. Soc. Lond. A 123, 714 (1929). Jensen, F. Introduction to Computational Chemistry 3rd edn (Wiley, 2017). Shavitt, I.&amp; Bartlett, R. J. Many-Body Methods in Chemistry and Physics. MBPT and Coupled-Cluster Theory (Cambridge Univ. Press, 2009). Mihm, T. N. et al. Nat. Comput. Sci.(2021). Liao, K.&amp; Greneis, A. J. Chem. Phys.145, 141102 (2016). Gruber, T., Liao, K., Tsatsoulis, T., Hummel, F.&amp; Gruneis, A. Phys. Rev. X 8, 021043 (2018). Kent, P. R. C.&amp; Kotliar, G. Science 361, 348 (2018). Knizia, G.&amp; Chan, G. K.-L. Phys. Rev. Lett.109, 186404 (2012). Perdew J.&amp; Kurth, S. A Primer in Density Functional Theory (Springer, 2003). Bogojeski, M., Vogt-Maranto, L., Tuckerman, M., Muller, K.&amp; Burke, K. Nat. Commun.11, 5223 (2020).',
 'Q1: Who first proposed the Dirac equation and when was it published?',
 'A1: The Dirac equation was first proposed by P.A.M. Dirac and published in the Proceedings of the Royal Society A in 1929, according to the ref

In [15]:
lisence_types=[raw_data_list[i]["Lisence"] for i in range(len(raw_data_list))]
lisence_types=list(set(lisence_types))

ng_words=[
"Creative Commons license (HIV-1 capsid)",
#"need to obtain permission ",
#"any author accepted manuscript version arising from this submission",
"notpermitted by statutory regulation ",
"Additionally, the package is availa",
]

filt_lisence_types=[]
for l in lisence_types:
    ng_flag=False
    for ng in ng_words:
        if ng in l:
            ng_flag=True
            break
    if ng_flag==False:
        filt_lisence_types.append(l)

filt_lisence_types=list(set(filt_lisence_types))
#lisence_typesをtxtで保存
with open("database/processed/lisence_types.txt","w") as f:
    for l in filt_lisence_types:
        f.write(l+"\n")

In [16]:
filt_raw_data_list=[]
for idx in range(len(raw_data_list)):
    lis=raw_data_list[idx]["Lisence"]
    add_flag=False
    if lis in filt_lisence_types:
        filt_raw_data_list.append(raw_data_list[idx])
    else:
        continue

len(filt_raw_data_list),lis

(65325,
 'Creative Commons licence and your intended use is not permitted by statutory regulation or exceeds the permitted use, you will need to obtain permission directly from the copyright holder. ')

In [17]:
#jsonとして出力
with open("database/output/refs0926.json","w") as f:
    json.dump(filt_raw_data_list,f,indent=4)

In [18]:
import re
import itertools
from tqdm import tqdm
def clean_text(text):
    pattern = r'\.(?=[A-Za-z])'
    text= re.sub(pattern, '. ', text)
    #連続したピリオドを削除
    text=re.sub(r'\.{2,}', '.', text)
    return text

context_ix=0

original_text_list=[]

for context_ix in tqdm(range(len(raw_data_list))):
    original_text=raw_data_list[context_ix]["main"]
    #flatten list
    original_text= list(itertools.chain.from_iterable(original_text))
    original_text="\n".join(original_text)
    original_text=clean_text(original_text)

       
    original_text_list.append(original_text)

100%|██████████| 65329/65329 [00:28<00:00, 2319.71it/s]


In [19]:
#context filt
context_list=[]
for idx in tqdm(range(len(q_a_list2))):
    targt_context=q_a_list2[idx][0]
    context_list.append(targt_context)

context_list=list(set(context_list))


100%|██████████| 419957/419957 [00:00<00:00, 4334327.43it/s]


In [20]:
import string

def starts_with_punctuation(text):
    if not text:
        return False
    return text[0] in string.punctuation

ng_context=[
"Scheme",
"$$",
"Fig.",
"{",
"&",
"You are a scientific",
]

filt_context_list=[]
for c in context_list:
    ng_flag=False

    for ng in ng_context:
        if ng in c:
            ng_flag=True
            break
    if len(c)<100:
        ng_flag=True

    if starts_with_punctuation(c):
        ng_flag=True
    
    if not c[0].isupper():
        ng_flag=True

    if not ng_flag:
        filt_context_list.append(c)

filt_context_list=sorted(filt_context_list)
with open("database/output/context0926.json","w") as f:
    json.dump(filt_context_list,f,indent=4)
len(filt_context_list)

56119

In [21]:

#q_a_with_context_list=[]
with open("database/output/qa0926.json","r") as f:
    q_a_with_context_list=json.load(f)

len(q_a_with_context_list)

180184

In [22]:
import numpy as np
context_array=np.array(filt_context_list)

#途中から
restart_id=284064+24290+5782
q_a_list3=q_a_list2[restart_id:]

def remove_id(q):
    q=q.split(":")[1:]
    q=":".join(q)
    q=q.strip()
    return q

for idx in tqdm(range(len(q_a_list3))):
    targt_context=q_a_list3[idx][0]

    if targt_context not in filt_context_list:
        continue

    for context_ix in (range(len(original_text_list))):
        pos=original_text_list[context_ix].find(targt_context[:30])
        if pos>0:
            q=remove_id(q_a_list3[idx][1])
            a=remove_id(q_a_list3[idx][2])
            
            context_id=np.where(targt_context==context_array)[0][0]
            d={
                "context_id":int(context_id),
               # "context":targt_context,
                "question":q,
                "answer":a,
                "ref_id":int(context_ix),
            }
            q_a_with_context_list.append(d)
            break


100%|██████████| 105821/105821 [9:44:03<00:00,  3.02it/s]  


In [23]:
#target_contextを書き出し
#jsonとして出力
with open("database/output/qa0926.json","w") as f:
    json.dump(q_a_with_context_list,f,indent=1)